<a href="https://colab.research.google.com/github/VivekVinushanth/TamilSongsLyricsCorpus/blob/master/Tamilsongs_lyric_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
!pip install scrapy

In [0]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [0]:
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class LyricSpider(scrapy.Spider):
    name = "tamil_lyrics"
    allowed_domains = ["tamilpaa.com"]
    start_urls = [
        # 'https://www.tamilpaa.com/tamil-movies-list-2020',
        'https://www.tamilpaa.com/tamil-movies-list-2019',
        'https://www.tamilpaa.com/tamil-movies-list-2018',
        'https://www.tamilpaa.com/tamil-movies-list-2017',
  
    ]
    base_url = 'https://www.tamilpaa.com/'
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'tamil_movies_lyric_all.json' ,                    # Used for pipeline 2
        'FEED_EXPORT_ENCODING' : 'utf-8'
    }
    
    # collecting movie url from starting pages
    def parse(self, response):
        for movie_url in response.xpath('//div[@class="image-overlay"]/a/@href'):
          # print("movie_url: ")
          # print(movie_url.extract())
          movie_url = movie_url.extract()
          yield scrapy.Request(movie_url,callback=self.parse_all_songs)

    # collecting all songs url from movie page
    def parse_all_songs(self,movie_url_response):
        for all_song_url in movie_url_response.xpath('//div[@class= "tab-content clearfix"]/ul/li/a/@href'):
            # print("all_song_url: ")
            # print(all_song_url.extract())
            all_song_url = all_song_url.extract()
            yield scrapy.Request(all_song_url,callback=self.parse_lyric)

    # # reaching particular song and scrap details
    # def parse_every_lyric(self,all_song_url_response):
    #     for lyric_url in all_song_url_response.xpath('//div[@class="tabs-container bordered-all"]/li/a/@href'):
    #         yield scrapy.Request(lyric_url, callback=self.parse_lyric)

    def parse_lyric(self, response):
            # print("parsing lyric......")
            movie = response.xpath(
                 '//table[@class="standard mb-10px"]/tbody/tr[1]/td[1]/text()').extract()
            # print(movie)
            movie = ' '.join([str(elem) for elem in movie])

            lyricist = response.xpath(
                '//table[@class="standard mb-10px"]/tbody/tr[2]/td[2]/text()').extract()
            # print(lyricist)
            lyricist = ' '.join([str(elem) for elem in lyricist])

            composer = response.xpath(
                '//table[@class="standard mb-10px"]//tbody/tr[1]/td[2]/text()').extract()
            # print(composer)
            composer = ' '.join([str(elem) for elem in composer])

            song = response.xpath(
                '//div/h3[@class="colored-text-2"]/text()').extract()
            # print(song)
            song = ' '.join([str(elem) for elem in song])
            
            year = response.xpath(
                '//table[@class="standard mb-10px"]/tbody/tr[2]/td[1]/text()').extract()
            # print(year)
            year = ' '.join([str(elem) for elem in year])

            singers = response.xpath(
                '//table[@class="standard mb-10px"]/tbody/tr[3]/td/text()').extract()
            # print(singers)
            singers = ','.join([str(elem) for elem in singers])

            lyrics = response.xpath('//div[@class="info-box white-bg"]/text()').extract()
            lyrics = ' '.join([str(elem) for elem in lyrics])
            lyrics=lyrics.replace('\n','')

            parsed_lyric =  {
            "Movie": movie,
            "Lyricist": lyricist,
            "Composer": composer,
            "Song": song,
            "Year": year,
            "Singers": singers,
            "lyric": lyrics
            }

            with open('/content/drive/My Drive/Lyrics_Others/'+song+'.json', 'w', encoding='utf8') as json_file:
              # json.dump(person_dict, json_file)
              json.dump(parsed_lyric,json_file,ensure_ascii=False)
                
            yield parsed_lyric

In [0]:
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class LyricSpider(scrapy.Spider):
    name = "tamil_lyrics"
    allowed_domains = ["tamilpaa.com"]
    start_urls = [
        'https://www.tamilpaa.com/tamil-movies-list',
    ]
    base_url = 'https://www.tamilpaa.com/'
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'tamil_movies_lyric_all.json' ,                    # Used for pipeline 2
        'FEED_EXPORT_ENCODING' : 'utf-8'
    }
    
    # collecting movie url from starting pages
    def parse(self, response):
        tables = response.xpath('//div[@class="one-col mb-5px"]/table[@class="standard mb-50px"]/tr/td[1]/a/@href')
        # print(tables)
        for movie_url in tables[1:]:
          # print("movie_url: ")
          # print(movie_url.extract())
          movie_url = movie_url.extract()
          yield scrapy.Request(movie_url,callback=self.parse_all_songs)

    # collecting all songs url from movie page
    def parse_all_songs(self,movie_url_response):
        for all_song_url in movie_url_response.xpath('//div[@class= "tab-content clearfix"]/ul/li/a/@href'):
            # print("all_song_url: ")
            # print(all_song_url.extract())
            all_song_url = all_song_url.extract()
            yield scrapy.Request(all_song_url,callback=self.parse_lyric)

    # # reaching particular song and scrap details
    # def parse_every_lyric(self,all_song_url_response):
    #     for lyric_url in all_song_url_response.xpath('//div[@class="tabs-container bordered-all"]/li/a/@href'):
    #         yield scrapy.Request(lyric_url, callback=self.parse_lyric)

    def parse_lyric(self, response):
            # print("parsing lyric......")
            movie = response.xpath(
                 '//table[@class="standard mb-10px"]/tbody/tr[1]/td[1]/text()').extract()
            # print(movie)
            movie = ' '.join([str(elem) for elem in movie])

            lyricist = response.xpath(
                '//table[@class="standard mb-10px"]/tbody/tr[2]/td[2]/text()').extract()
            # print(lyricist)
            lyricist = ' '.join([str(elem) for elem in lyricist])

            composer = response.xpath(
                '//table[@class="standard mb-10px"]//tbody/tr[1]/td[2]/text()').extract()
            # print(composer)
            composer = ' '.join([str(elem) for elem in composer])

            song = response.xpath(
                '//div/h3[@class="colored-text-2"]/text()').extract()
            # print(song)
            song = ' '.join([str(elem) for elem in song])
            
            year = response.xpath(
                '//table[@class="standard mb-10px"]/tbody/tr[2]/td[1]/text()').extract()
            # print(year)
            year = ' '.join([str(elem) for elem in year])

            singers = response.xpath(
                '//table[@class="standard mb-10px"]/tbody/tr[3]/td/text()').extract()
            # print(singers)
            singers = ','.join([str(elem) for elem in singers])

            lyrics = response.xpath('//div[@class="info-box white-bg"]/text()').extract()
            lyrics = ' '.join([str(elem) for elem in lyrics])
            lyrics=lyrics.replace('\n','')

            parsed_lyric =  {
            "Movie": movie,
            "Lyricist": lyricist,
            "Composer": composer,
            "Song": song,
            "Year": year,
            "Singers": singers,
            "lyric": lyrics
            }

            with open('/content/drive/My Drive/Lyrics_Others/'+song+'.json', 'w', encoding='utf8') as json_file:
              # json.dump(person_dict, json_file)
              json.dump(parsed_lyric,json_file,ensure_ascii=False)
                
            yield parsed_lyric

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(LyricSpider)
process.start()

2020-06-12 20:41:45 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-06-12 20:41:45 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
2020-06-12 20:41:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-06-12 20:41:45 [scrapy.crawler] INFO: Overridden settings:
{'FEED_EXPORT_ENCODING': 'utf-8',
 'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
/usr/local/lib/python3.6/dist-packages/scrapy/extensions/feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)
